In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

# Pkg.add("CoordinateTransformations")
# Pkg.add("RigidBodyDynamics")
# Pkg.add("MeshCatMechanisms")
# Pkg.add("MeshCat")

using CoordinateTransformations: Translation
using RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Printf
using LinearAlgebra

  Activating project at `~/cmu/courses/16745_OCRL/zoe2_optimal_controller/julia`


In [2]:
vis = Visualizer()
# open(vis)  # open the visualizer in a separate tab/window
render(vis) # render the visualizer here inside the jupyter notebook


┌ Info: Listening on: 127.0.0.1:8700, thread id: 1
└ @ HTTP.Servers /home/hayden/.julia/packages/HTTP/4AUPl/src/Servers.jl:382
┌ Info: MeshCat server started. You can open the visualizer by visiting the following URL in your browser:
│ http://127.0.0.1:8700
└ @ MeshCat /home/hayden/.julia/packages/MeshCat/9QrxD/src/visualizer.jl:43


MeshCat.DisplayedVisualizer(MeshCat.CoreVisualizer(MeshCat.SceneTrees.SceneNode(nothing, nothing, Dict{String, Vector{UInt8}}(), nothing, Dict{String, MeshCat.SceneTrees.SceneNode}()), Set{HTTP.WebSockets.WebSocket}(), ip"127.0.0.1", 8700, HTTP.Servers.Server{HTTP.Servers.Listener{Nothing, Sockets.TCPServer}}(HTTP.Servers.Listener{Nothing, Sockets.TCPServer}(Sockets.InetAddr{Sockets.IPv4}(ip"127.0.0.1", 8700), "127.0.0.1", "8700", nothing, Sockets.TCPServer(RawFD(22) active)), nothing, Set{HTTP.Connections.Connection}(), Task (runnable) @0x00007741160b5140, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (0, 0, 142463702892))), MeshCat.AnimationContext[]))

In [3]:
urdf_path = joinpath(@__DIR__, "zoe2.urdf")
robot = parse_urdf(urdf_path)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf_path), vis)

MechanismVisualizer{MechanismState{Float64, Float64, Float64, TypeSortedCollections.TypeSortedCollection{Tuple{Vector{Joint{Float64, Revolute{Float64}}}}, 1}}, Visualizer}(MechanismState{Float64, Float64, Float64, …}(…), MeshCat Visualizer with path /meshcat at http://127.0.0.1:8700, 12)

In [4]:
# Create a state for the mechanism
state = MechanismState(robot)
q = configuration(state)

println("Initial configuration vector:")
println(q)
println("Number of degrees of freedom: ", length(q))

# Iterate over the joints to print their names
for (i, joint) in enumerate(joints(robot))
    println("Joint ", i, ": ", joint.name)
end

# Example: update a joint value
# q[5] += 1.0

# Update the visualizer with the new configuration
set_configuration!(mvis, q)

# state = MechanismState(robot, randn(2), randn(2))
# t, q, v = simulate(state, 5.0);

Initial configuration vector:
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Number of degrees of freedom: 7
Joint 1: axle_roll_back_joint
Joint 2: axle_yaw_front_joint
Joint 3: axle_yaw_back_joint
Joint 4: wheel_front_right_joint
Joint 5: wheel_front_left_joint
Joint 6: wheel_back_right_joint
Joint 7: wheel_back_left_joint


In [ ]:
include(joinpath(@__DIR__, "zoe2.jl"))

# Model parameters
model = (
    L = 1.0, # robot length
    B = 0.5, # robot width
    r = 0.1  # wheel radius
)

# Simulation parameters
params = (
    dt = 0.1, # seconds
    total_steps = 2,
    model = model
)

# Define our states
x = 0.0
y = 0.0
ψ = 0.0
θ_f = 0.01   # front steering angle (radians)
θ_r = 0.0   # rear steering angle (radians)

# Each state vector: [x_b, y_b, ψ, θ_f, θ_r]
Xsim = [zeros(5) for _ in 1:params.total_steps]
Xsim[1] = [x, y, ψ, θ_f, θ_r]

# Define our control parameters (start as straight motion)
ω_fl = 0.1
ω_fr = 0.1
ω_rl = 0.1
ω_rr = 0.1

# Each control vector: [ω_fl, ω_fr, ω_rl, ω_rr]
Usim = [zeros(4) for _ in 1:params.total_steps]
Usim[1] = [ω_fl, ω_fr, ω_rl, ω_rr]

# Compute the changes in the states based on the control and accumulate over steps
for i in 1:params.total_steps-1

    println("Xsim[", i, "]: ", Xsim[i])
    println("Usim[", i, "]: ", Usim[i])

    Xdot = zoe2_dynamics(params.model, Xsim[i], Usim[i])
    # println("Xdot: ", Xdot)

    # Unpack the dynamics vector
    dx_b = Xdot[1]
    dy_b = Xdot[2]
    dψ = Xdot[3]
    dθ_f = Xdot[4]
    dθ_r = Xdot[5]

    # Update the state based on the dynamics
    x += dx_b * params.dt
    y += dy_b * params.dt
    ψ += dψ * params.dt
    θ_f += dθ_f * params.dt
    θ_r += dθ_r * params.dt

    # Set the next vector
    Xsim[i+1] = [x, y, ψ, θ_f, θ_r]
    Usim[i+1] = [ω_fl, ω_fr, ω_rl, ω_rr]
end

# println("Simulation data:")
# println("x_b, y_b, ψ, θ_f, θ_r")
# for i in 1:params.total_steps
#     @printf("%3d: %8.2f, %8.2f, %8.2f, %8.2f, %8.2f\n", i, Xsim[i][1], Xsim[i][2], Xsim[i][3], Xsim[i][4], Xsim[i][5])
# end

# animate_zoe2(Xsim, dt)


Xsim[1]: [0.0, 0.0, 0.01, 0.0, 0.0]
Usim[1]: [0.1, 0.1, 0.1, 0.1]
R_b: 500000.0
R_fl: 1.0e6
R_fr: 1.0e6
R_rl: 1.0e6
R_rr: 1.0e6
v_b: 0.0
ψ_dot: 0.0
θ_f_dot: 0.0
θ_r_dot: 0.0
x_dot: 0.0
y_dot: 0.0
